# Out-of-site run times
Version of the out-of-site ablation tests that gets the run-times for the final model and architecture ablation tests.
#### Notes
- Only one fold model is created for each run
- Only 20 runs per test - i.e. timings are for one ensemble of twenty runs.
- No parallel running of tests
- The first test (test0) is the proposed model, so other tests are offset by 1 compared to the full ablation test (e.g. the dropout test is test4 here and test3 in the ablation test)
- By default, training times are output to the `train_stats.csv` file, so these could also be obtained from the ablation tests model directories. This notebook allows testing under more controlled conditions - e.g. if the full runs are run in a shared environment they may not be accurate/consistent due to the server workload. So this is a cut-down version that can run in a small dedicated environment.

In [ ]:
import os
import pandas as pd

import initialise
import common
from modelling_functions import create_models, run_experiment
from architecture_out_of_site import model_params
from model_parameters import ModelParams, ExperimentParams
import scenarios

## Input files
Change these settings as required
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `prism_csv`: The file containing extracted PRISM data for each sample, created by `Extract PRISM Data.ipynb`
- `aux_csv`: The file containing extracted sample labels, DEM, climate zone and other auxiliary data, created by `Extract Auxiliary Data.ipynb`.

In [ ]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_365days.csv')
prism_csv = os.path.join(common.DATASETS_DIR, 'prism_365days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_365days.csv')

## Set up experiment parameters
If the experiment dictionary contains a 'tests' key that is not 'falsy' (False, None, 0, empty list) it is assumed to be a list of tests to run. Each test will run with the specified model parameters. Model parameters not specified will be the same for each test, as set in the main model_params dictionary. A failed run can be restarted by setting the 'restart' key to the test that failed. This test and the remaining tests will then be run.

If 'tests' is 'falsy' then a single test will be run using the parameters in the main model_params dictionary.

In [ ]:
experiment = ExperimentParams({
    'name': 'out-of-site_timings',
    'description': 'Timings for out-of-site architecture changes',
    'tests': [
        {'testName': 'Proposed model'},
        {'testName': 'Conv filters: 32',
         'blocks': {
             'opticalConv': [{'filters': 32}, {'filters': 32}, {'filters': 32}],
             'weatherConv': [{'filters': 32}, {'filters': 32}, {'filters': 32}]
             },
        },
        {'testName': 'Dense layers: 2', 'blocks': {'fc': [{'units': 128}, {'units': 128}]}},
        {'testName': 'Dense units: 256', 'blocks': {'fc': [{'units': 256}]}},
        {'testName': 'Dropout: 0.5', 'dropoutRate': 0.5},
        {'testName': 'Batch size: 32', 'batchSize': 32},
        {'testName': 'Epochs: 100', 'epochs': 100},
    ],
    'restart': None,
})

experiment

## Set up model parameters

### Model parameters settings
To find out more about any parameter, run `model_params.help('<parameter>')`. 

In [ ]:
# Customize model parameters
scenarios.out_of_site_scenario(model_params)
model_params['modelName'] = experiment['name']
model_params['description'] = experiment['description']
model_params['samplesFile'] = aux_csv
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])
model_params['splitFolds'] = 0
model_params['testSize'] = 0.1

model_params.add_input('optical', {'filename': modis_csv, 'channels': 7})
model_params.add_input('weather', {'filename': prism_csv, 'channels': 7})

model_params

## Build and run the models
Builds and trains the LFMC models. After training each model, several derived models are created and evaluated. The full list of models is:
- `base` - The fully trained model
- `best` - A model using the checkpoint with the best training loss
- `merge10` - A model created by merging the last 10 checkpoints. The checkpoints are merged by averaging the corresponding weights from each model.
- `ensemble10` - An ensembled model of the last 10 checkpoints. This model averages the predictions made by each model in the ensemble to make the final prediction.
- `merge_best10` - Similar to the merge10 model, but uses the 10 checkpoints with the lowest training/validation losses.

All models, predictions, evaluation statistics, and plots of test results are saved to `model_dir`, with each test and run saved to a separate sub-directory. For each model created, predictions and evaluation statistics are also returned as attributes of the `model` object. These are stored as nested lists, the structure for a full experiment is:
- Tests (omitted if not an experiment)
  - Runs (omitted for a single run)
    - Folds (for k-fold splitting)

In [ ]:
models = run_experiment(experiment, model_params)
for model in models:
    display(getattr(model, 'test_stats', None))

## Display the training times
Time are Tensorflow/Keras model training time and excludes data preparation time.

In [ ]:
train_times = []
for test in models:
    run_time = 0
    for run in test:
        df = pd.read_csv(os.path.join(run.model_dir, 'train_stats.csv'))
        run_time += df.trainTime[0]
    weights = df.trainableWeights[0]
    train_times.append([run_time/60, run_time/60/len(test), weights])
pd.DataFrame(train_times, index=experiment['testNames'],
             columns=['ensemble_time', 'single_time', 'num_params']).round(2)